In [1]:
import pandas as pd
import numpy as np
import csv
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection, ensemble, preprocessing, metrics

In [2]:
data=pd.read_csv("python_data1.csv",engine='python')

In [3]:
data.head(2)

,code,date,GPM,OPM,NIM,EBITDA,cash,account_receviable,inventory,fixed_asset,invest,liq_asset,liq_liability,leverage,revenue,mkt_value,GOR
0,1101,200803,13.91,9.89,9.53,4041764,12423117,7631637,7302870,115287797,-2650659,57091512,29783960,46.86,17906537,191344000.0,-0.1019
1,1101,200806,12.17,8.28,11.23,8364504,11760556,9501174,6841077,119771536,-7462419,54362283,42792997,51.40,38460834,133194000.0,0.0683


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25960 entries, 0 to 25959
Data columns (total 17 columns):
code                  25960 non-null int64
date                  25960 non-null int64
GPM                   25960 non-null float64
OPM                   25960 non-null float64
NIM                   25960 non-null float64
EBITDA                25960 non-null int64
cash                  25960 non-null int64
account_receviable    25960 non-null int64
inventory             25960 non-null int64
fixed_asset           25960 non-null int64
invest                25960 non-null int64
liq_asset             25960 non-null int64
liq_liability         25960 non-null int64
leverage              25960 non-null float64
revenue               25960 non-null int64
mkt_value             25960 non-null float64
GOR                   25960 non-null float64
dtypes: float64(6), int64(11)
memory usage: 3.4 MB


########## 改欄名
########## data.columns = [...]

########## 資料合併 
########## pd.concat ( [A,B] ,axis=1 )

# 【觀察資料、敘述統計、偏態、轉整數】

In [5]:
data['EV'] = data['mkt_value'] + data['liq_liability'] - data['cash']
data['EBITDA_EV'] = (data['EBITDA'] / data['EV'])*1000   # 0.1% 單位
data['GOR'] = data['GOR']*100

In [6]:
data['EBITDA_EV_lag'] = data['EBITDA_EV'].shift(+1)
data['change'] = data['EBITDA_EV']-data['EBITDA_EV_lag']

In [7]:
data.head()

,code,date,GPM,OPM,NIM,EBITDA,cash,account_receviable,inventory,fixed_asset,...,liq_asset,liq_liability,leverage,revenue,mkt_value,GOR,EV,EBITDA_EV,EBITDA_EV_lag,change
0,1101,200803,13.91,9.89,9.53,4041764,12423117,7631637,7302870,115287797,...,57091512,29783960,46.86,17906537,191344000.0,-10.19,208704843.0,19.365933,NaN,NaN
1,1101,200806,12.17,8.28,11.23,8364504,11760556,9501174,6841077,119771536,...,54362283,42792997,51.40,38460834,133194000.0,6.83,164226441.0,50.932748,19.365933,31.566815
2,1101,200809,11.21,7.83,9.30,11993727,10615331,9283486,8550177,122733210,...,45692556,39599056,52.06,60944205,61399000.0,10.40,90382725.0,132.699329,50.932748,81.766581
3,1101,200812,9.90,6.28,8.25,15111414,13033535,8386187,6908626,125807120,...,43762069,39174857,51.70,78476558,88724000.0,2.39,114865322.0,131.557669,132.699329,-1.141661
4,1101,200903,9.73,5.77,2.69,2829888,14200950,9431949,5539320,127230295,...,48506002,38714520,51.81,18071221,92016000.0,0.92,116529570.0,24.284720,131.557669,-107.272949


# ．使用 R 做補值
##  ---> 類似R的IF  :  np.where(data['A']>=30,1,0)   
##  --->  敘述統計   : data.describe()                     
##  --->      偏態      :  data.skew(axis=0)                    

In [8]:
data=pd.read_csv("python_data2.csv",engine='python')

In [9]:
data = data.astype(int)
data['label'] = np.where(data['change']>=30,1,0)

In [10]:
data.describe()

,code,date,GPM,OPM,NIM,EBITDA,cash,account_receviable,inventory,fixed_asset,...,liq_liability,leverage,revenue,mkt_value,GOR,EV,EBITDA_EV,EBITDA_EV_lag,change,label
count,25960.000000,25960.000000,25960.000000,25960.000000,25960.000000,2.596000e+04,2.596000e+04,2.596000e+04,2.596000e+04,2.596000e+04,...,2.596000e+04,25960.000000,2.596000e+04,2.596000e+04,25960.000000,2.596000e+04,25960.000000,25960.000000,25960.000000,25960.000000
mean,3366.784284,201257.500000,19.193413,-7.795185,1.429545,1.877395e+06,4.359460e+06,4.545351e+06,4.301350e+06,9.354451e+06,...,1.076097e+07,42.571726,1.636215e+07,2.200790e+07,52.925270,2.791307e+07,57.502465,57.509707,-0.186133,0.277735
std,2351.025044,287.253248,18.725125,401.746241,678.636725,8.046860e+06,2.277929e+07,2.318394e+07,1.682119e+07,3.217948e+07,...,4.785069e+07,17.557707,7.519858e+07,8.074474e+07,2894.147997,9.842146e+07,100.636994,100.642570,106.869658,0.447891
min,1101.000000,200803.000000,-557.000000,-32157.000000,-22185.000000,-2.146405e+07,4.840000e+02,0.000000e+00,0.000000e+00,0.000000e+00,...,1.475000e+03,0.000000,-1.916478e+09,8.500000e+04,-99.000000,-1.729697e+09,-3196.000000,-3196.000000,-4667.000000,0.000000
25%,1733.000000,201008.250000,9.000000,0.000000,0.000000,8.800350e+04,3.387805e+05,3.124900e+05,3.721892e+05,6.244542e+05,...,8.713890e+05,30.000000,1.101516e+06,2.214750e+06,-11.000000,2.938812e+06,21.000000,21.000000,-18.000000,0.000000
50%,2464.000000,201257.500000,17.000000,4.000000,4.000000,3.250075e+05,9.045880e+05,9.186610e+05,9.416095e+05,1.659830e+06,...,2.221060e+06,43.000000,3.048604e+06,5.188500e+06,0.000000,6.818246e+06,48.000000,48.000000,15.000000,0.000000
75%,3481.000000,201506.750000,27.000000,10.000000,9.000000,1.009981e+06,2.414978e+06,2.547179e+06,2.772695e+06,4.976073e+06,...,6.163521e+06,55.000000,8.930965e+06,1.243225e+07,13.000000,1.636509e+07,88.000000,88.000000,32.000000,1.000000
max,9958.000000,201712.000000,100.000000,160.000000,81135.000000,2.891660e+08,7.550959e+08,1.229237e+09,5.609549e+08,4.663030e+08,...,2.025502e+09,102.000000,2.045071e+09,2.027462e+09,417465.000000,2.078569e+09,5802.000000,5802.000000,5763.000000,1.000000


In [11]:
data.skew(axis=0)

code                    1.575811
date                    0.000000
GPM                    -5.194147
OPM                   -54.849269
NIM                    77.035320
EBITDA                 14.442787
cash                   21.175347
account_receviable     20.561184
inventory              15.891328
fixed_asset             7.945374
invest                -11.958904
liq_asset              16.998355
liq_liability          18.068881
leverage                0.072254
revenue                 7.855768
mkt_value               9.680757
GOR                   120.939992
EV                      8.804429
EBITDA_EV              10.405664
EBITDA_EV_lag          10.403909
change                  2.702741
label                   0.992573
dtype: float64

# 【迴圈---以code做預測】

In [12]:
# 建立 code 清單
code_list=pd.DataFrame(data['code'].unique())
code_list.columns=['code']

In [13]:
# 建立空資料  #是個 DataFrame
result = pd.DataFrame()    

In [14]:
for ix in range(0,code_list.size):
    #創立新的空資料。code清單。
    result1 = pd.DataFrame()  
    pre_data = data[data['code']==code_list.iloc[ix,0]]
    
    # train、test
    train = pre_data[pre_data['date']<=201512]
    test = pre_data[pre_data['date']>201512]
    
    select_column = ['GPM','OPM','NIM','EBITDA','cash','account_receviable','inventory','fixed_asset','invest','liq_asset','liq_liability','leverage','revenue','mkt_value','GOR','EV','EBITDA_EV','change']
    x_train = train[select_column]
    y_train = train['label']
    x_test = test[select_column]
    y_test = test['label']
    
    # 使用隨機森林預測
    rfc = RandomForestClassifier() 
    rfc.fit(x_train, y_train)
    EBITDA_EV_predict = rfc.predict(x_test)

    # 績效
    accuracy = metrics.accuracy_score(y_test, EBITDA_EV_predict)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, EBITDA_EV_predict)
    auc = metrics.auc(fpr, tpr)
    
    #紀錄結果
    # EBITDA_EV_predict
    apple = pd.DataFrame(EBITDA_EV_predict)
    result1 = result1.append(apple)
    # code、AUC
    result1['code'] = code_list.iloc[ix,0]
    result1['AUC'] = accuracy
    result1 = result1.loc[[0,4],:]
    
    #合併資料
    result = result.append(result1)
        

C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
 

C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
 

C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
 

C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
C:\Users\User\program\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
 

In [15]:
result.columns = ['label','code','AUC']

In [16]:
result

,label,code,AUC
0,0,1101,1.000
4,0,1101,1.000
0,0,1102,0.500
4,0,1102,0.500
0,0,1103,1.000
4,0,1103,1.000
0,0,1104,0.875
4,0,1104,0.875
0,0,1108,1.000
4,0,1108,1.000


###  ---> .loc[  [特定列數]  ,:] 
### --->  .sort_values(['要降冪排序的欄位'],ascending=False) 

In [17]:
# 選2016、2017第一季
pre_2016 = result.loc[[0],:]
pre_2017 = result.loc[[4],:]

In [18]:
pre_2016 = pre_2016.sort_values(['label','AUC'],ascending=False)
pre_2017 = pre_2016.sort_values(['label','AUC'],ascending=False)

In [20]:
pre_2016.to_csv('pre_2016.csv')
pre_2017.to_csv('pre_2017.csv')

In [ ]:
pre_2016

## 【迴圈前測試】

In [427]:
code_list.iloc[2,0]

1103

In [428]:
# pre_data
pre_data = data[data['code']==code_list.iloc[2,0]]

## ．train資料 、text資料（ 201512區分）

In [429]:
train = pre_data[pre_data['date']<=201512]
test = pre_data[pre_data['date']>201512]
#text = text[text['date']<=201612]

In [430]:
select_column = ['GPM','OPM','NIM','EBITDA','cash','account_receviable','inventory','fixed_asset','invest','liq_asset','liq_liability','leverage','revenue','mkt_value','GOR','EV','EBITDA_EV','change']

x_train = train[select_column]
y_train = train['label']
x_test = test[select_column]
y_test = test['label']

## ．使用隨機森林預測

In [431]:
# 建立模組
rfc = RandomForestClassifier() 
# 訓練模組
rfc.fit(x_train, y_train)
# 預測
EBITDA_EV_predict = rfc.predict(x_test)
EBITDA_EV_predict

array([0, 0, 0, 1, 0, 0, 0, 1])

In [432]:
# 績效
accuracy = metrics.accuracy_score(y_test, EBITDA_EV_predict)
fpr, tpr, thresholds = metrics.roc_curve(y_test, EBITDA_EV_predict)
auc = metrics.auc(fpr, tpr)
#print(auc)
print('準確率: {}'.format(auc))
print('AUC值: {}'.format(accuracy))

準確率: 0.8333333333333333
AUC值: 0.875


## ．紀錄結果

In [433]:
# EBITDA_EV_predict
apple = pd.DataFrame(EBITDA_EV_predict)

In [434]:
result1 = result1.append(apple)

C:\Users\User\program\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:87: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


In [435]:
# code、AUC
result1['code'] = code_list.iloc[2,0]
result1['AUC'] = accuracy

In [436]:
result = result.append(result1)

In [437]:
result

,0,code,AUC
0,0,1102,0.875
1,0,1102,0.875
2,0,1102,0.875
3,1,1102,0.875
4,0,1102,0.875
5,0,1102,0.875
6,1,1102,0.875
7,1,1102,0.875
0,0,1103,0.875
1,0,1103,0.875
